In [29]:
import torch
import numpy as np
import os
import pickle
import argparse
import matplotlib.pyplot as plt
from copy import deepcopy
from itertools import repeat
from tqdm import tqdm
from einops import rearrange
import wandb
import time
from torchvision import transforms
import h5py
import cv2 as cv
from brl_constants import FPS
from brl_constants import PUPPET_GRIPPER_JOINT_OPEN
from brl_constants import TASK_CONFIGS
from utils import load_data  # data functions
from utils import sample_box_pose, sample_insertion_pose  # robot functions
from utils import (
    compute_dict_mean,
    set_seed,
    detach_dict,
    calibrate_linear_vel,
    postprocess_base_action,
)  # helper functions
from policy import ACTPolicy, CNNMLPPolicy

# from policy import ACTPolicy, CNNMLPPolicy, DiffusionPolicy
from visualize_episodes import save_videos

from detr.models.latent_model import Latent_Model_Transformer
import argparse

In [30]:
task_name = "act_demo_z1_push_red"
task_config = TASK_CONFIGS[task_name]
camera_names = task_config["camera_names"]

ckpt_dir = "/mnt/data1/act/act_demo_z1_push_red/ckpt"
policy_class = "ACT"
kl_weight = 10
chunk_size = 100
hidden_dim = 512
batch_size = 8
dim_feedforward = 3200
num_steps = 2000
lr = 1e-5
lr_backbone = 1e-5
seed = 0
backbone = "resnet18"
state_dim = 6
enc_layers = 4
dec_layers = 7
nheads = 8

policy_config = {
    "lr": lr,
    "num_queries": chunk_size,
    "kl_weight": kl_weight,
    "hidden_dim": hidden_dim,
    "dim_feedforward": dim_feedforward,
    "lr_backbone": lr_backbone,
    "backbone": backbone,
    "enc_layers": enc_layers,
    "dec_layers": dec_layers,
    "nheads": nheads,
    "camera_names": camera_names,
    "vq": False,
    "vq_class": None,
    "vq_dim": None,
    "action_dim": 6,
    "no_encoder": False,
}

In [31]:
def get_args_parser():
    parser = argparse.ArgumentParser("Set transformer detector", add_help=False)
    parser.add_argument("--lr", default=1e-4, type=float)  # will be overridden
    parser.add_argument("--lr_backbone", default=1e-5, type=float)  # will be overridden
    parser.add_argument("--batch_size", default=2, type=int)  # not used
    parser.add_argument("--weight_decay", default=1e-4, type=float)
    parser.add_argument("--epochs", default=300, type=int)  # not used
    parser.add_argument("--lr_drop", default=200, type=int)  # not used
    parser.add_argument(
        "--clip_max_norm",
        default=0.1,
        type=float,  # not used
        help="gradient clipping max norm",
    )

    # Model parameters
    # * Backbone
    parser.add_argument(
        "--backbone",
        default="resnet18",
        type=str,  # will be overridden
        help="Name of the convolutional backbone to use",
    )
    parser.add_argument(
        "--dilation",
        action="store_true",
        help="If true, we replace stride with dilation in the last convolutional block (DC5)",
    )
    parser.add_argument(
        "--position_embedding",
        default="sine",
        type=str,
        choices=("sine", "learned"),
        help="Type of positional embedding to use on top of the image features",
    )
    parser.add_argument(
        "--camera_names",
        default=[],
        type=list,  # will be overridden
        help="A list of camera names",
    )

    # * Transformer
    parser.add_argument(
        "--enc_layers",
        default=4,
        type=int,  # will be overridden
        help="Number of encoding layers in the transformer",
    )
    parser.add_argument(
        "--dec_layers",
        default=6,
        type=int,  # will be overridden
        help="Number of decoding layers in the transformer",
    )
    parser.add_argument(
        "--dim_feedforward",
        default=2048,
        type=int,  # will be overridden
        help="Intermediate size of the feedforward layers in the transformer blocks",
    )
    parser.add_argument(
        "--hidden_dim",
        default=256,
        type=int,  # will be overridden
        help="Size of the embeddings (dimension of the transformer)",
    )
    parser.add_argument(
        "--dropout", default=0.1, type=float, help="Dropout applied in the transformer"
    )
    parser.add_argument(
        "--nheads",
        default=8,
        type=int,  # will be overridden
        help="Number of attention heads inside the transformer's attentions",
    )
    parser.add_argument(
        "--num_queries",
        default=400,
        type=int,  # will be overridden
        help="Number of query slots",
    )
    parser.add_argument("--pre_norm", action="store_true")

    # * Segmentation
    parser.add_argument(
        "--masks",
        action="store_true",
        help="Train segmentation head if the flag is provided",
    )

    # repeat args in imitate_episodes just to avoid error. Will not be used
    parser.add_argument("--eval", action="store_true")
    parser.add_argument("--onscreen_render", action="store_true")
    parser.add_argument(
        "--ckpt_dir", action="store", type=str, help="ckpt_dir", required=False
    )
    parser.add_argument(
        "--policy_class",
        action="store",
        type=str,
        help="policy_class, capitalize",
        required=False,
    )
    parser.add_argument(
        "--task_name", action="store", type=str, help="task_name", required=False
    )
    parser.add_argument("--seed", action="store", type=int, help="seed", required=False)
    parser.add_argument(
        "--num_steps", action="store", type=int, help="num_epochs", required=False
    )
    parser.add_argument(
        "--kl_weight", action="store", type=int, help="KL Weight", required=False
    )
    parser.add_argument(
        "--chunk_size", action="store", type=int, help="chunk_size", required=False
    )
    parser.add_argument("--temporal_agg", action="store_true")

    parser.add_argument("--use_vq", action="store_true")
    parser.add_argument(
        "--vq_class", action="store", type=int, help="vq_class", required=False
    )
    parser.add_argument(
        "--vq_dim", action="store", type=int, help="vq_dim", required=False
    )
    parser.add_argument("--load_pretrain", action="store_true", default=False)
    parser.add_argument("--action_dim", action="store", type=int, required=False)
    parser.add_argument(
        "--eval_every",
        action="store",
        type=int,
        default=500,
        help="eval_every",
        required=False,
    )
    parser.add_argument(
        "--validate_every",
        action="store",
        type=int,
        default=500,
        help="validate_every",
        required=False,
    )
    parser.add_argument(
        "--save_every",
        action="store",
        type=int,
        default=500,
        help="save_every",
        required=False,
    )
    parser.add_argument(
        "--resume_ckpt_path",
        action="store",
        type=str,
        help="load_ckpt_path",
        required=False,
    )
    parser.add_argument("--no_encoder", action="store_true")
    parser.add_argument("--skip_mirrored_data", action="store_true")
    parser.add_argument(
        "--actuator_network_dir",
        action="store",
        type=str,
        help="actuator_network_dir",
        required=False,
    )
    parser.add_argument("--history_len", action="store", type=int)
    parser.add_argument("--future_len", action="store", type=int)
    parser.add_argument("--prediction_len", action="store", type=int)

    return parser

In [32]:
parser = argparse.ArgumentParser(
    "DETR training and evaluation script", parents=[get_args_parser()]
)
args = parser.parse_args(["--policy_class", "ACT"])

In [33]:
config_path = "/mnt/data1/act/act_demo_z1_push_red/ckpt/config.pkl"
with open(config_path, "rb") as f:
    policy_config = pickle.load(f)
policy_config

{'num_steps': 2000,
 'eval_every': 500,
 'validate_every': 500,
 'save_every': 500,
 'ckpt_dir': '/mnt/data1/act/act_demo_z1_push_red/ckpt',
 'resume_ckpt_path': None,
 'episode_len': 400,
 'state_dim': 6,
 'lr': 1e-05,
 'policy_class': 'ACT',
 'onscreen_render': False,
 'policy_config': {'lr': 1e-05,
  'num_queries': 100,
  'kl_weight': 10,
  'hidden_dim': 512,
  'dim_feedforward': 3200,
  'lr_backbone': 1e-05,
  'backbone': 'resnet18',
  'enc_layers': 4,
  'dec_layers': 7,
  'nheads': 8,
  'camera_names': ['wrist'],
  'vq': False,
  'vq_class': None,
  'vq_dim': None,
  'action_dim': 6,
  'no_encoder': False},
 'task_name': 'act_demo_z1_push_red',
 'seed': 0,
 'temporal_agg': False,
 'camera_names': ['wrist'],
 'real_robot': True,
 'load_pretrain': False,
 'actuator_config': {'actuator_network_dir': None,
  'history_len': None,
  'future_len': None,
  'prediction_len': None}}

In [34]:
policy_config = {
    "lr": 1e-05,
    "num_queries": 100,
    "kl_weight": 10,
    "hidden_dim": 512,
    "dim_feedforward": 3200,
    "lr_backbone": 1e-05,
    "backbone": "resnet18",
    "enc_layers": 4,
    "dec_layers": 7,
    "nheads": 8,
    "camera_names": ["wrist"],
    "vq": False,
    "vq_class": None,
    "vq_dim": None,
    "action_dim": 6,
    "no_encoder": False,
    "task_name": "act_demo_z1_push_red",
    "ckpt_dir": "/mnt/data1/act/act_demo_z1_push_red/ckpt/policy_best.ckpt",
    "num_steps": 2000,
    "lr": 1e-5,
    "seed": 0,
    "policy_class": "ACT",
}

In [35]:
# ckpt_path = "/mnt/data1/act/act_demo_z1_push_red/ckpt/policy_best.ckpt"

policy = ACTPolicy(policy_config)
loading_status = policy.deserialize(torch.load(policy_config["ckpt_dir"]))
print(loading_status)
policy.cuda()
policy.eval()
print("Loaded: ", policy_config["ckpt_dir"])
stats_path = "/mnt/data1/act/act_demo_z1_push_red/ckpt/dataset_stats.pkl"
with open(stats_path, "rb") as f:
    stats = pickle.load(f)

DETR Args:  Namespace(lr=0.0001, lr_backbone=1e-05, batch_size=2, weight_decay=0.0001, epochs=300, lr_drop=200, clip_max_norm=0.1, backbone='resnet18', dilation=False, position_embedding='sine', camera_names=[], enc_layers=4, dec_layers=6, dim_feedforward=2048, hidden_dim=256, dropout=0.1, nheads=8, num_queries=400, pre_norm=False, masks=False, eval=False, onscreen_render=False, ckpt_dir='/mnt/data1/act/act_demo_z1_push_red/ckpt', policy_class='ACT', task_name='act_demo_z1_push_red', seed=0, num_steps=2000, kl_weight=None, chunk_size=None, temporal_agg=False, use_vq=False, vq_class=None, vq_dim=None, load_pretrain=False, action_dim=None, eval_every=500, validate_every=500, save_every=500, resume_ckpt_path=None, no_encoder=False, skip_mirrored_data=False, actuator_network_dir=None, history_len=None, future_len=None, prediction_len=None)
ACT Args:  Namespace(lr=1e-05, lr_backbone=1e-05, batch_size=2, weight_decay=0.0001, epochs=300, lr_drop=200, clip_max_norm=0.1, backbone='resnet18', di

/home/zfei/anaconda3/envs/torch/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zfei/anaconda3/envs/torch/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Use VQ: False, None, None
number of parameters: 83.91M
KL Weight 10
<All keys matched successfully>
Loaded:  /mnt/data1/act/act_demo_z1_push_red/ckpt/policy_best.ckpt


In [64]:
pre_process_qpos = (
    lambda s_qpos: torch.from_numpy((s_qpos - stats["qpos_mean"]) / stats["qpos_std"])
    .float()
    .cuda()
    .unsqueeze(0)
)
post_process_action = lambda a: a * stats["action_std"] + stats["action_mean"]
query_frequency = policy_config["num_queries"]
BASE_DELAY = 13
query_frequency -= BASE_DELAY
max_timesteps = int(400)  # may increase for real-world tasks


def pre_proccess_img(img):
    img_torch = torch.from_numpy(img).unsqueeze(0)
    img_torch = torch.einsum("k h w c -> k c h w", img_torch)
    img_torch = (img_torch / 255.0).float().cuda().unsqueeze(0)
    return img_torch

In [62]:
with torch.inference_mode():
    time0 = time.time()
    DT = 1 / FPS
    culmulated_delay = 0
    for t in range(max_timesteps):
        all_actions = policy(qpos, curr_image)

NameError: name 'curr_image' is not defined

In [69]:
h5data_file = "/mnt/data1/act/act_demo_z1_push_red/episode_50.hdf5"
with torch.inference_mode():
    with h5py.File(h5data_file, "r") as root:
        tic = time.time()
        for index in range(100):
            qpos_np = root["/observations/qpos"][index]
            img_np = root["/observations/images/wrist"][index]
            qpos = pre_process_qpos(qpos_np)
            curr_img = pre_proccess_img(img_np)
            # print("qpos: ",qpos.shape)
            # print("curr_img: ",curr_img.shape)
            all_actions = policy(qpos, curr_img)
            # print("all_actions: ", all_actions.shape)
        print("ave inference time:", (time.time()-tic)/100)
        print("ave fps:", 100/(time.time()-tic))

ave inference time: 0.008824615478515626
ave fps: 113.31442555046989


In [72]:
h5data_file = "/mnt/data1/act/act_demo_z1_push_red/episode_50.hdf5"
with torch.inference_mode():
    with h5py.File(h5data_file, "r") as root:
        tic = time.time()
        for index in range(100):
            qpos_np = root["/observations/qpos"][index]
            img_np = root["/observations/images/wrist"][index]
            qpos = pre_process_qpos(qpos_np)
            curr_img = pre_proccess_img(img_np)
            # print("qpos: ",qpos.shape)
            # print("curr_img: ",curr_img.shape)
            all_actions = policy(qpos, curr_img)
            # print("all_actions: ", all_actions.shape)

            raw_action = all_actions[:, 0]
            raw_action = raw_action.squeeze(0).cpu().numpy()
            actual_action = post_process_action(raw_action)
            print("actual action %i"%index, actual_action)

        print("ave inference time:", (time.time()-tic)/100)

actual action 0 [-0.27048644  0.28409767 -1.3577642   1.0764028   0.26963568  0.00364211]
actual action 1 [-0.27048752  0.28410238 -1.357761    1.0763901   0.2696367   0.00364215]
actual action 2 [-0.2705621   0.2836768  -1.3582773   1.0780164   0.26963714  0.00363334]
actual action 3 [-0.27061105  0.2891059  -1.3531578   1.0689373   0.27101272  0.00433012]
actual action 4 [-0.27016604  0.31722903 -1.3378539   1.0374017   0.27179506  0.00484861]
actual action 5 [-0.27019146  0.31718796 -1.3378454   1.0374027   0.27181014  0.0048494 ]
actual action 6 [-0.27219585  0.3273734  -1.3273525   1.0072116   0.27330372  0.00488721]
actual action 7 [-0.27810234  0.35023183 -1.3025774   0.9570044   0.27953658  0.00487137]
actual action 8 [-0.281964    0.36514717 -1.2856654   0.92124915  0.28307167  0.00488848]
actual action 9 [-0.28510687  0.3823722  -1.269188    0.88054186  0.28775796  0.00490073]
actual action 10 [-0.2853763   0.38249648 -1.2684586   0.87936634  0.28799316  0.00490519]
actual ac

In [54]:
curr_img.shape

torch.Size([1, 3, 480, 640])

In [44]:
pre_process(qpos_np)

tensor([[ 0.6369, -1.5069, -1.7223,  1.7077, -0.6148,  0.0204]],
       device='cuda:0')

In [ ]:
with torch.inference_mode():
    time0 = time.time()
    DT = 1 / FPS
    culmulated_delay = 0
    all_actions = policy(qpos, curr_image)